
# <p style="padding:10px;background-color:#ffff2b;margin:10;color:#0000FF;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px;overflow:hidden;font-weight:30">Book Recommender System</p>

In [1]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# from SQL-SERVER DATABASE to load the data
import pyodbc

# Replace with your SQL Server details
server = 'NAGESH\SQLEXPRESS'
database = 'ML_Projects_db'
username = 'sa'
password = 'nagesh@sharvan'

# Connect to SQL Server
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};'
connection = pyodbc.connect(connection_string)


In [4]:
# Execute SQL Query
query = 'SELECT * FROM sys.tables'
data = pd.read_sql(query, connection)

In [5]:
data

,name,object_id,principal_id,schema_id,parent_object_id,type,type_desc,create_date,modify_date,is_ms_shipped,...,history_retention_period_unit_desc,is_node,is_edge,data_retention_period,data_retention_period_unit,data_retention_period_unit_desc,ledger_type,ledger_type_desc,ledger_view_id,is_dropped_ledger_table
0,books_data,1269579561,None,1,0,U,USER_TABLE,2024-01-04 16:58:56.310,2024-01-04 16:58:56.310,False,...,None,False,False,-1,-1,INFINITE,0,NON_LEDGER_TABLE,None,False


In [6]:
# Execute SQL Query

# table = 'books_data'    # Change this to the desired table name
query = 'SELECT * FROM books_data'
data = pd.read_sql(query, connection)

In [7]:
data

,Title,Author,Genre,SubGenre,Height,Publisher
0,Fundamentals of Wavelets,"Goswami, Jaideva",tech,signal_processing,228,Wiley
1,Data Smart,"Foreman, John",tech,data_science,235,Wiley
2,God Created the Integers,"Hawking, Stephen",tech,mathematics,197,Penguin
3,Superfreakonomics,"Dubner, Stephen",science,economics,179,HarperCollins
4,Orientalism,"Said, Edward",nonfiction,history,197,Penguin
...,...,...,...,...,...,...
206,Structure and Randomness,"Tao, Terence",science,mathematics,252,None
207,Image Processing with MATLAB,"Eddins, Steve",tech,signal_processing,241,None
208,Animal Farm,"Orwell, George",fiction,classic,180,None
209,"Idiot, The","Dostoevsky, Fyodor",fiction,classic,197,None


In [8]:
# copy the data from original data
df = data.copy()

In [9]:
df.head()

,Title,Author,Genre,SubGenre,Height,Publisher
0,Fundamentals of Wavelets,"Goswami, Jaideva",tech,signal_processing,228,Wiley
1,Data Smart,"Foreman, John",tech,data_science,235,Wiley
2,God Created the Integers,"Hawking, Stephen",tech,mathematics,197,Penguin
3,Superfreakonomics,"Dubner, Stephen",science,economics,179,HarperCollins
4,Orientalism,"Said, Edward",nonfiction,history,197,Penguin


In [10]:
df.drop('Height', axis=1, inplace=True)

In [11]:
df.head(2)

,Title,Author,Genre,SubGenre,Publisher
0,Fundamentals of Wavelets,"Goswami, Jaideva",tech,signal_processing,Wiley
1,Data Smart,"Foreman, John",tech,data_science,Wiley


In [12]:
df.isnull().sum()

Title         0
Author       24
Genre         0
SubGenre      0
Publisher    96
dtype: int64

In [13]:
df.dropna(inplace=True)

In [14]:
df.isnull().sum()

Title        0
Author       0
Genre        0
SubGenre     0
Publisher    0
dtype: int64

In [15]:
# Combine relevant text columns into a single 'text' column
df['tags'] = df['Author'] + ' ' + df['Genre'] + ' ' + df['SubGenre'] + ' ' + df['Publisher']

In [16]:
df.head()

,Title,Author,Genre,SubGenre,Publisher,tags
0,Fundamentals of Wavelets,"Goswami, Jaideva",tech,signal_processing,Wiley,"Goswami, Jaideva tech signal_processing Wiley"
1,Data Smart,"Foreman, John",tech,data_science,Wiley,"Foreman, John tech data_science Wiley"
2,God Created the Integers,"Hawking, Stephen",tech,mathematics,Penguin,"Hawking, Stephen tech mathematics Penguin"
3,Superfreakonomics,"Dubner, Stephen",science,economics,HarperCollins,"Dubner, Stephen science economics HarperCollins"
4,Orientalism,"Said, Edward",nonfiction,history,Penguin,"Said, Edward nonfiction history Penguin"


In [17]:
df['tags'][0]

'Goswami, Jaideva tech signal_processing Wiley'

In [18]:
df['tags'].apply(lambda x : x.lower())

0          goswami, jaideva tech signal_processing wiley
1                  foreman, john tech data_science wiley
2              hawking, stephen tech mathematics penguin
3        dubner, stephen science economics harpercollins
4                said, edward nonfiction history penguin
                             ...                        
114              sen, amartya science economics springer
115    huntington, samuel nonfiction history simon&sc...
116          capra, fritjof nonfiction anthology fontana
117          bach, richard nonfiction autobiography dell
119                  braithwaite fiction classic penguin
Name: tags, Length: 112, dtype: object

In [19]:
df['tags'] = df['tags'].apply(lambda x : x.lower())

In [20]:
df.head(3)

,Title,Author,Genre,SubGenre,Publisher,tags
0,Fundamentals of Wavelets,"Goswami, Jaideva",tech,signal_processing,Wiley,"goswami, jaideva tech signal_processing wiley"
1,Data Smart,"Foreman, John",tech,data_science,Wiley,"foreman, john tech data_science wiley"
2,God Created the Integers,"Hawking, Stephen",tech,mathematics,Penguin,"hawking, stephen tech mathematics penguin"


In [21]:
# df.to_csv('recommend_book.csv', index=False)

In [22]:
df['tags'][1]

'foreman, john tech data_science wiley'

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words="english")

In [24]:
vectors = cv.fit_transform(df['tags']).toarray()

In [25]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [26]:
vectors[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int64)

In [27]:
# !pip3 install --upgrade scikit-learn

In [28]:
cv.get_feature_names_out

<bound method CountVectorizer.get_feature_names_out of CountVectorizer(max_features=5000, stop_words='english')>

In [29]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [30]:
# !pip3 uninstall scikit-learn
# !pip3 install --upgrade nltk

In [31]:
# !pip3 list

In [32]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [33]:
ps.stem('sucssesfully')

'sucsses'

In [34]:
stem('foreman, john tech data_science wiley')

'foreman, john tech data_sci wiley'

In [35]:
df['tags'].apply

<bound method Series.apply of 0          goswami, jaideva tech signal_processing wiley
1                  foreman, john tech data_science wiley
2              hawking, stephen tech mathematics penguin
3        dubner, stephen science economics harpercollins
4                said, edward nonfiction history penguin
                             ...                        
114              sen, amartya science economics springer
115    huntington, samuel nonfiction history simon&sc...
116          capra, fritjof nonfiction anthology fontana
117          bach, richard nonfiction autobiography dell
119                  braithwaite fiction classic penguin
Name: tags, Length: 112, dtype: object>

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

In [37]:
similarity = cosine_similarity(vectors)

In [38]:
similarity

array([[1.       , 0.4      , 0.2      , ..., 0.       , 0.       ,
        0.       ],
       [0.4      , 1.       , 0.2      , ..., 0.       , 0.       ,
        0.       ],
       [0.2      , 0.2      , 1.       , ..., 0.       , 0.       ,
        0.2236068],
       ...,
       [0.       , 0.       , 0.       , ..., 1.       , 0.2      ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.2      , 1.       ,
        0.       ],
       [0.       , 0.       , 0.2236068, ..., 0.       , 0.       ,
        1.       ]])

In [39]:
similarity.shape

(112, 112)

In [40]:
similarity[0]

array([1.        , 0.4       , 0.2       , 0.        , 0.        ,
       0.2       , 0.        , 0.        , 0.4       , 0.        ,
       0.2       , 0.        , 0.        , 0.18257419, 0.        ,
       0.        , 0.18257419, 0.2       , 0.        , 0.        ,
       0.2       , 0.2       , 0.2       , 0.2       , 0.2       ,
       0.18257419, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.2       , 0.2       , 0.2236068 , 0.4       , 0.18257419,
       0.4       , 0.4       , 0.18257419, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [41]:
sorted(similarity[0], reverse=True)

[0.9999999999999999,
 0.39999999999999997,
 0.39999999999999997,
 0.39999999999999997,
 0.39999999999999997,
 0.39999999999999997,
 0.22360679774997896,
 0.22360679774997896,
 0.19999999999999998,
 0.19999999999999998,
 0.19999999999999998,
 0.19999999999999998,
 0.19999999999999998,
 0.19999999999999998,
 0.19999999999999998,
 0.19999999999999998,
 0.19999999999999998,
 0.19999999999999998,
 0.19999999999999998,
 0.18257418583505539,
 0.18257418583505539,
 0.18257418583505539,
 0.18257418583505539,
 0.18257418583505539,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,

In [42]:
sorted(list(enumerate(similarity[0])), reverse=True, key= lambda x:x[1])[1:6]

[(1, 0.39999999999999997),
 (8, 0.39999999999999997),
 (58, 0.39999999999999997),
 (60, 0.39999999999999997),
 (61, 0.39999999999999997)]

In [43]:
def recommend(book):
    book_index = df[df['Title'] == book].index[0]
    distances = similarity[book_index]
    books_list = sorted(list(enumerate(distances)), reverse=True, key= lambda x:x[1])[1:6]
    
    for i in books_list:
        print(df.iloc[i[0]].Title)

In [44]:
recommend('God Created the Integers')

Drunkard's Walk, The
Analysis, Vol I
Signal and the Noise, The
Freakonomics
Theory of Everything, The
